# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Sockets") \
    .master("spark://fc91669459e3:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 13:42:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Ejemplo con el formato "rate"

In [3]:
rate_df = spark \
            .readStream \
            .format("rate") \
            .load()

rate_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



In [4]:
query = rate_df.filter(rate_df.value % 2 == 1) \
                .writeStream \
                .outputMode("append") \
                .format("console") \
                .start()

25/03/25 13:57:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-57bfebc6-7742-4b7e-8ccd-58c29d465384. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/25 13:57:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+
|           timestamp|value|
+--------------------+-----+
|2025-03-25 13:57:...|    1|
|2025-03-25 13:57:...|    3|
+--------------------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+
|           timestamp|value|
+--------------------+-----+
|2025-03-25 13:57:...|    5|
|2025-03-25 13:57:...|    7|
|2025-03-25 13:57:...|    9|
+--------------------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+-----+
|           timestamp|value|
+--------------------+-----+
|2025-03-25 13:57:...|   11|
+--------------------+-----+

--------------

In [8]:
query.stop()

### Ejemplo con el formato "socket"

In [14]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

lines = spark \
            .readStream \
            .format("socket") \
            .option("host", "localhost") \
            .option("port", "9999") \
            .load()

25/03/25 14:22:22 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [ ]:
from pyspark.sql.functions import explode, split

# Simple processing example: word counter
words = lines.select(explode(split(lines.value, " ")).alias("word"))
word_count = words.groupBy("word").count()

query = word_count \
        .writeStream \
        .outputMode("complete") \
        .format("console") \
        .start()

query.awaitTermination(15)

25/03/25 14:22:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ecfefcce-ae75-406f-a499-93d38adc6f4a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/25 14:22:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/03/25 14:22:36 ERROR MicroBatchExecution: Query [id = d3b455f4-4d34-4f00-b1b5-4eade10fd549, runId = 3e5cd52b-aabb-4a49-a89b-dfb1e1697212] terminated with error
java.net.ConnectException: Connection refused
	at java.base/sun.nio.ch.Net.connect0(Native Method)
	at java.base/sun.nio.ch.Net.connect(Net.java:579)
	at java.base/sun.nio.ch.Net.connect(Net.java:568)
	at java.base/sun.nio.ch.NioSocketImpl.connect(NioSocketImpl.java:593)
	at java.base/java.net.SocksSocketImpl.connect(SocksSocketImpl.java:327)
	at java.base/java.net.Socket.connect(Socket.java:633)
	at java.base/java.net.Socket.connect(Socket.java:583)
	at java.base/java.net.Socket.<init>(Socket.java:507)
	at java.base/java.net.Socket.<init>(Socket.java:287)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchStream.initialize(TextSocketMicroBatchStream.scala:71)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchStream.planInputPartitions(TextSocketMicroBatchStream.scala:117)
	at org.

In [12]:
query.stop()

In [ ]:
sc.stop()